In [2]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine, text as sql_text
from datetime import datetime, timedelta
import openpyxl
from scipy.stats import zscore
from scipy.stats import norm
from pandas.tseries.offsets import DateOffset

In [1]:
rename_dict = {
    'umsatz': 'revenue',
    'umsatz-1': 'revenue-1',
    'umsatz-2': 'revenue-2',
    'umsatz-3': 'revenue-3',
    'umsatzVJ': 'revenuePY',
    'umsatzVJ-1': 'revenuePY-1',
    'umsatzVJ-2': 'revenuePY-2',
    'umsatzVJ-3': 'revenuePY-3',
    'umsatzVJ+2': 'revenuePY+2',
    'umsatzVJ+1': 'revenuePY+1',
    'umsatz+2': 'revenue+2',
    'karneval': 'carnival',
    'ostern': 'easter',
    'himmelfahrt': 'ascension_day',
    'pfingsten': 'whitsunday',
    'weihnachten': 'christmas',
    'neujahr': 'new_year'
}

In [2]:
def get_previous_year_monday(row):
    # Extrahiere Jahr und Kalenderwoche aus der 'kalenderwoche'-Spalte
    week, year = map(int, row['kalenderwoche'].split('-'))
    year -= 1  # Vorjahr

    # Finde den ersten Montag des Vorjahres
    first_day_of_year = datetime(year, 1, 1)

    # Berechne den ersten Montag des Vorjahres
    first_monday = first_day_of_year + timedelta(days=(7 - first_day_of_year.weekday())) if first_day_of_year.weekday() != 0 else first_day_of_year

    # Berechne den Montag der entsprechenden Kalenderwoche des Vorjahres
    previous_year_monday = first_monday + timedelta(weeks=week - 1)
    return previous_year_monday


In [3]:
# Funktion zur Berechnung des Medcouple (MC)
def medcouple(x):
    x = np.sort(x)
    n = len(x)
    med = np.median(x)
    below_med = x[x < med]
    above_med = x[x > med]
    
    if len(below_med) == 0 or len(above_med) == 0:
        return 0
    
    pairwise_diff = (above_med[:, None] - below_med[None, :])
    medcouple_scores = ((above_med[:, None] - med) - (med - below_med[None, :])) / pairwise_diff
    
    return np.median(medcouple_scores[np.isfinite(medcouple_scores)])

# Funktion zur Berechnung der robusten Skala (MAD)
def robust_scale(y):
    med = y.median()
    mad = ((y - med).abs()).median() / norm.ppf(0.75)
    return mad

# Funktion zur Berechnung der Adjusted Outlyingness (AO) mit Pandas
def adjusted_outlyingness(group, column):
    y = group[column]
    med = y.median()
    mc = medcouple(y)
    
    # Berechnung der Quartile und des IQR
    Q1 = y.quantile(0.25)
    Q3 = y.quantile(0.75)
    IQR = Q3 - Q1

    # Anpassung der Whisker basierend auf MC
    if mc > 0:
        lower_whisker = Q1 - 1.5 * np.exp(-4 * mc) * IQR
        upper_whisker = Q3 + 1.5 * np.exp(3 * mc) * IQR
    else:
        lower_whisker = Q1 - 1.5 * np.exp(-3 * mc) * IQR
        upper_whisker = Q3 + 1.5 * np.exp(4 * mc) * IQR
    
    # Berechnung der AO-Werte
    AO = pd.Series(index=y.index, dtype=float)
    AO[y >= med] = (y[y >= med] - med) / (upper_whisker - med)
    AO[y < med] = (med - y[y < med]) / (med - lower_whisker)
    
    # Identifizierung der Ausreißer basierend auf den angepassten Whiskern
    
    group['is_outlier'] = (y < lower_whisker) | (y > upper_whisker)
    group['AO'] = AO
    
    return group

**Laden der Daten**

In [4]:
datei_pfad = 'bakery_data.csv'
df_roh = pd.read_csv(datei_pfad)
df_roh = df_roh[df_roh['kd_plz'].notna()]
df_roh = df_roh[df_roh['umsatz'].notna()]
df_roh['kd_plz'] = df_roh['kd_plz'].astype(int)
df_roh['datum'] = pd.to_datetime(df_roh['datum'])

**Add Bundesland**

In [5]:
df_bd = pd.read_csv("../data/PLZ_BD_Mapping.csv",sep=";")

In [6]:
df_bd = df_bd.rename({'PLZ' : 'kd_plz', 'BUNDESLAND' : 'Bundesland'}, axis = 1)
df_bd['kd_plz'] = df_bd['kd_plz'].astype(int)

In [7]:
df_bd = df_bd.groupby('kd_plz')['Bundesland'].agg(lambda x: x.mode().iloc[0]).reset_index()

In [8]:
df_roh= df_roh.merge(df_bd, on = 'kd_plz', how = 'left')

**Add Vorjahr**

In [9]:
df_no_shift = df_roh.copy()
df_no_shift['datumVJ'] = df_no_shift.apply(get_previous_year_monday, axis=1)

In [10]:

kalender = pd.read_excel("kalender.xlsx")
kalender_nrw = pd.read_excel("kalender_NRW.xlsx")
kalender_nds = pd.read_excel("kalender_NDS.xlsx")
kalender_rp= pd.read_excel("kalender_RP.xlsx")
kalender_bawü = pd.read_excel("kalender_BaWü.xlsx")
kalender_he = pd.read_excel("kalender_HE.xlsx")
kalender_scho = pd.read_excel("kalender_ScHo.xlsx")

kalender = kalender[['datum', 'datumVJ']]
kalender_nrw = kalender_nrw[['datum', 'datumVJ']]
kalender_nds = kalender_nds[['datum', 'datumVJ']]
kalender_rp = kalender_rp[['datum', 'datumVJ']]
kalender_bawü = kalender_bawü[['datum', 'datumVJ']]
kalender_he = kalender_he[['datum', 'datumVJ']]
kalender_scho = kalender_scho[['datum', 'datumVJ']]

kalender['datum'] = pd.to_datetime(kalender['datum'], format='%d.%m.%Y')
kalender['datumVJ'] = pd.to_datetime(kalender['datumVJ'], format='%d.%m.%Y')

kalender_nrw['datum'] = pd.to_datetime(kalender_nrw['datum'], format='%d.%m.%Y')
kalender_nrw['datumVJ'] = pd.to_datetime(kalender_nrw['datumVJ'], format='%d.%m.%Y')

kalender_nds['datum'] = pd.to_datetime(kalender_nds['datum'], format='%d.%m.%Y')
kalender_nds['datumVJ'] = pd.to_datetime(kalender_nds['datumVJ'], format='%d.%m.%Y')

kalender_he['datum'] = pd.to_datetime(kalender_he['datum'], format='%d.%m.%Y')
kalender_he['datumVJ'] = pd.to_datetime(kalender_he['datumVJ'], format='%d.%m.%Y')

kalender_rp['datum'] = pd.to_datetime(kalender_rp['datum'], format='%d.%m.%Y')
kalender_rp['datumVJ'] = pd.to_datetime(kalender_rp['datumVJ'], format='%d.%m.%Y')

kalender_bawü['datum'] = pd.to_datetime(kalender_bawü['datum'], format='%d.%m.%Y')
kalender_bawü['datumVJ'] = pd.to_datetime(kalender_bawü['datumVJ'], format='%d.%m.%Y')

kalender_scho['datum'] = pd.to_datetime(kalender_scho['datum'], format='%d.%m.%Y')
kalender_scho['datumVJ'] = pd.to_datetime(kalender_scho['datumVJ'], format='%d.%m.%Y')


In [11]:

dataframes = {
    'Nordrhein-Westfalen': kalender_nrw,
    'Hessen': kalender_he,
    'Rheinland-Pfalz': kalender_rp,
    'Niedersachsen': kalender_nds,
    'Schleswig-Holstein': kalender_scho,
    'Baden-Württemberg': kalender_bawü,
    
}
merged_df_list = []

# Iterieren Sie über die einzigartigen Bundesländer im Haupt-DataFrame
for bundesland in df_roh['Bundesland'].unique():
    if bundesland in dataframes:
        # Filter für das aktuelle Bundesland
        df_filtered = df_roh[df_roh['Bundesland'] == bundesland]
        # Merge mit dem entsprechenden DataFrame
        merged = pd.merge(df_filtered, dataframes[bundesland], on='datum', how='left')
        # Hinzufügen zum Ergebnis-DataFrame
        merged_df_list.append(merged)

# Zusammenfügen aller Teil-DataFrames
df_school_shift = pd.concat(merged_df_list, ignore_index=True)

In [12]:
df_public_shift = pd.merge(df_roh, kalender, on ='datum', how = 'left')

**Add umsatzVJ for Analyse**

In [13]:
df_no_shift = df_no_shift.merge(df_no_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=['datumVJ', 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', 'VJ'))

In [14]:
df_public_shift = df_public_shift.merge(df_public_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=['datumVJ', 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', 'VJ'))

In [15]:
df_school_shift = df_school_shift.merge(df_school_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=['datumVJ', 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', 'VJ'))

In [16]:
df_no_shift = df_no_shift.loc[:, ~df_no_shift.columns.duplicated()]

In [17]:
df_public_shift = df_public_shift.loc[:, ~df_public_shift.columns.duplicated()]

In [18]:
df_school_shift = df_school_shift.loc[:, ~df_school_shift.columns.duplicated()]

**Safe as .csv**

In [19]:
df_no_shift.to_csv('no_shift_roh.csv', index=False)
df_no_shift = df_no_shift.drop(['umsatzVJ'], axis=1)

In [20]:
df_public_shift.to_csv('public_shift_roh.csv', index=False)
df_public_shift = df_public_shift.drop(['umsatzVJ'], axis=1)

In [21]:
df_school_shift.to_csv('school_shift_roh.csv', index=False)
df_school_shift = df_school_shift.drop(['umsatzVJ'], axis=1)

**Outlier (AO)**

In [22]:
df_no_shift = df_no_shift.groupby(['datum'], group_keys=False).apply(adjusted_outlyingness, column = 'umsatz').reset_index(drop=True)
df_no_shift = df_no_shift[df_no_shift['is_outlier'] == False]

C:\Users\aaron\AppData\Local\Temp\ipykernel_3384\175811738.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_no_shift = df_no_shift.groupby(['datum'], group_keys=False).apply(adjusted_outlyingness, column = 'umsatz').reset_index(drop=True)


In [23]:
df_public_shift = df_public_shift.groupby(['datum'], group_keys=False).apply(adjusted_outlyingness, column = 'umsatz').reset_index(drop=True)
df_public_shift = df_public_shift[df_public_shift['is_outlier'] == False]

C:\Users\aaron\AppData\Local\Temp\ipykernel_3384\3390694159.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_public_shift = df_public_shift.groupby(['datum'], group_keys=False).apply(adjusted_outlyingness, column = 'umsatz').reset_index(drop=True)


In [24]:
df_school_shift = df_school_shift.groupby(['datum'], group_keys=False).apply(adjusted_outlyingness, column = 'umsatz').reset_index(drop=True)
df_school_shift = df_school_shift[df_school_shift['is_outlier'] == False]

C:\Users\aaron\AppData\Local\Temp\ipykernel_3384\1532561050.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_school_shift = df_school_shift.groupby(['datum'], group_keys=False).apply(adjusted_outlyingness, column = 'umsatz').reset_index(drop=True)


**Add UmsatzVJ fürs Model**

In [25]:
df_no_shift = df_no_shift.merge(df_no_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=['datumVJ', 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', 'VJ'))

In [26]:
df_public_shift = df_public_shift.merge(df_public_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=['datumVJ', 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', 'VJ'))

In [27]:
df_school_shift = df_school_shift.merge(df_school_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=['datumVJ', 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', 'VJ'))

In [28]:
df_no_shift = df_no_shift.drop(['datumVJ'], axis=1)

In [29]:
df_public_shift = df_public_shift.drop(['datumVJ'], axis=1)

In [30]:
df_school_shift = df_school_shift.drop(['datumVJ'], axis=1)

**Add other dates**

In [31]:
for offset in range(1, 4):
    
    new_column_name = f"datum-{offset}"
    df_no_shift[new_column_name] = df_no_shift["datum"] - pd.DateOffset(weeks=offset)
    
for offset in range(1, 3):
    
    new_column_name = f"datum+{offset}"
    df_no_shift[new_column_name] = df_no_shift["datum"] + pd.DateOffset(weeks=offset)
    

In [32]:
for offset in range(1, 4):
    
    new_column_name = f"datum-{offset}"
    df_public_shift[new_column_name] = df_public_shift["datum"] - pd.DateOffset(weeks=offset)
    
for offset in range(1, 3):
    
    new_column_name = f"datum+{offset}"
    df_public_shift[new_column_name] = df_public_shift["datum"] + pd.DateOffset(weeks=offset)
    

In [33]:
for offset in range(1, 4):
    
    new_column_name = f"datum-{offset}"
    df_school_shift[new_column_name] = df_school_shift["datum"] - pd.DateOffset(weeks=offset)
    
for offset in range(1, 3):
    
    new_column_name = f"datum+{offset}"
    df_school_shift[new_column_name] = df_school_shift["datum"] + pd.DateOffset(weeks=offset)
    

**Add umsatz**

In [34]:
for offset in range(1, 4):
    df_no_shift = df_no_shift.merge(df_no_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=[f"datum-{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"-{offset}"))
    df_no_shift = df_no_shift.loc[:,~df_no_shift.columns.duplicated()].copy()
    df_no_shift = df_no_shift.merge(df_no_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatzVJ']], left_on=[f"datum-{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"-{offset}"))
    df_no_shift = df_no_shift.loc[:,~df_no_shift.columns.duplicated()].copy()
    
for offset in range(1, 3):
    df_no_shift = df_no_shift.merge(df_no_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=[f"datum+{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"+{offset}"))
    df_no_shift = df_no_shift.loc[:,~df_no_shift.columns.duplicated()].copy()
    df_no_shift = df_no_shift.merge(df_no_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatzVJ']], left_on=[f"datum+{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"+{offset}"))
    df_no_shift = df_no_shift.loc[:,~df_no_shift.columns.duplicated()].copy()

In [35]:
for offset in range(1, 4):
    df_public_shift = df_public_shift.merge(df_public_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=[f"datum-{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"-{offset}"))
    df_public_shift = df_public_shift.loc[:,~df_public_shift.columns.duplicated()].copy()
    df_public_shift = df_public_shift.merge(df_public_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatzVJ']], left_on=[f"datum-{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"-{offset}"))
    df_public_shift = df_public_shift.loc[:,~df_public_shift.columns.duplicated()].copy()
    
for offset in range(1, 3):
    df_public_shift = df_public_shift.merge(df_public_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=[f"datum+{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"+{offset}"))
    df_public_shift = df_public_shift.loc[:,~df_public_shift.columns.duplicated()].copy()
    df_public_shift = df_public_shift.merge(df_public_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatzVJ']], left_on=[f"datum+{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"+{offset}"))
    df_public_shift = df_public_shift.loc[:,~df_public_shift.columns.duplicated()].copy()

In [36]:
for offset in range(1, 4):
    df_school_shift = df_school_shift.merge(df_school_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=[f"datum-{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"-{offset}"))
    df_school_shift = df_school_shift.loc[:,~df_school_shift.columns.duplicated()].copy()
    df_school_shift = df_school_shift.merge(df_school_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatzVJ']], left_on=[f"datum-{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"-{offset}"))
    df_school_shift = df_school_shift.loc[:,~df_school_shift.columns.duplicated()].copy()
    
for offset in range(1, 3):
    df_school_shift = df_school_shift.merge(df_school_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatz']], left_on=[f"datum+{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"+{offset}"))
    df_school_shift = df_school_shift.loc[:,~df_school_shift.columns.duplicated()].copy()
    df_school_shift = df_school_shift.merge(df_school_shift[['datum', 'filialnummer', 'Bäckerei', 'umsatzVJ']], left_on=[f"datum+{offset}", 'filialnummer', 'Bäckerei'], right_on=['datum', 'filialnummer', 'Bäckerei'], how='left', suffixes=('', f"+{offset}"))
    df_school_shift = df_school_shift.loc[:,~df_school_shift.columns.duplicated()].copy()

### **one hot encoded Variablen**

In [37]:
df_no_shift['datum+2'] = df_no_shift['datum+2'].astype(str)
df_public_shift['datum+2'] = df_public_shift['datum+2'].astype(str)
df_school_shift['datum+2'] = df_school_shift['datum+2'].astype(str)

In [38]:
# Beispiel-Listen mit Feiertagsdaten
karneval = ['2023-02-13', '2024-02-05', '2025-02-24']
ostern = ['2023-04-10', '2024-04-01', '2025-04-21']
himmelfahrt = ['2023-05-15', '2024-05-06', '2024-05-26']
pfingsten = ['2023-05-29', '2024-05-20', '2025-06-09']
weihnachten = ['2022-12-26', '2023-12-25', '2024-12-23', '2025-12-25']
neujahr = ['2023-02-01', '2024-01-01', '2025-03-01']


In [39]:
def dummy_variables(row, liste):
    if row['datum+2'] in liste:
        return True
    else:
        return False

In [40]:
df_no_shift['karneval'] = df_no_shift.apply(lambda row: dummy_variables(row, karneval), axis=1)
df_no_shift['ostern'] = df_no_shift.apply(lambda row: dummy_variables(row, ostern), axis=1)
df_no_shift['himmelfahrt'] = df_no_shift.apply(lambda row: dummy_variables(row, himmelfahrt), axis=1)
df_no_shift['pfingsten'] = df_no_shift.apply(lambda row: dummy_variables(row, pfingsten), axis=1)
df_no_shift['weihnachten'] = df_no_shift.apply(lambda row: dummy_variables(row, weihnachten), axis=1)
df_no_shift['neujahr'] = df_no_shift.apply(lambda row: dummy_variables(row, neujahr), axis=1)

In [41]:
df_public_shift['karneval'] = df_public_shift.apply(lambda row: dummy_variables(row, karneval), axis=1)
df_public_shift['ostern'] = df_public_shift.apply(lambda row: dummy_variables(row, ostern), axis=1)
df_public_shift['himmelfahrt'] = df_public_shift.apply(lambda row: dummy_variables(row, himmelfahrt), axis=1)
df_public_shift['pfingsten'] = df_public_shift.apply(lambda row: dummy_variables(row, pfingsten), axis=1)
df_public_shift['weihnachten'] = df_public_shift.apply(lambda row: dummy_variables(row, weihnachten), axis=1)
df_public_shift['neujahr'] = df_public_shift.apply(lambda row: dummy_variables(row, neujahr), axis=1)

In [42]:
df_school_shift['karneval'] = df_school_shift.apply(lambda row: dummy_variables(row, karneval), axis=1)
df_school_shift['ostern'] = df_school_shift.apply(lambda row: dummy_variables(row, ostern), axis=1)
df_school_shift['himmelfahrt'] = df_school_shift.apply(lambda row: dummy_variables(row, himmelfahrt), axis=1)
df_school_shift['pfingsten'] = df_school_shift.apply(lambda row: dummy_variables(row, pfingsten), axis=1)
df_school_shift['weihnachten'] = df_school_shift.apply(lambda row: dummy_variables(row, weihnachten), axis=1)
df_school_shift['neujahr'] = df_school_shift.apply(lambda row: dummy_variables(row, neujahr), axis=1)

**Safe data**

In [43]:
df_no_shift = df_no_shift.dropna()

In [44]:
df_public_shift = df_public_shift.dropna()

In [45]:
df_school_shift = df_school_shift.dropna()

**Add unique column**

In [46]:
df_no_shift['filiale_baeckerei'] = df_no_shift['filialnummer'].astype(str) + '_' + df_no_shift['Bäckerei']


In [47]:
df_public_shift['filiale_baeckerei'] = df_public_shift['filialnummer'].astype(str) + '_' + df_public_shift['Bäckerei']


In [48]:
df_school_shift['filiale_baeckerei'] = df_school_shift['filialnummer'].astype(str) + '_' + df_school_shift['Bäckerei']


In [ ]:
df_no_shift = df_no_shift.rename(columns=rename_dict)
df_public_shift = df_public_shift.rename(columns=rename_dict)
df_school_shift = df_school_shift.rename(columns=rename_dict)

In [51]:
df_no_shift.to_csv('no_shift_forecasting.csv', index=False)

In [52]:
df_public_shift.to_csv('public_shift_forecasting.csv', index=False)

In [53]:
df_school_shift.to_csv('school_shift_forecasting.csv', index=False)